In [1]:
%tensorflow_version 1.x
!pip install h5py==2.10.0
!pip install flask_ngrok
!pip install flask==1.1.1
!pip install itsdangerous==1.1.0
!pip install gunicorn==19.9.0
!pip install jinja2==2.11.3
!pip install matplotlib>=1.4.3
!pip install PyYAML>=3.11,<=4.2b1
!pip install pyngrok==4.1.1

TensorFlow 1.x selected.
     |████████████████████████████████| 2.9 MB 8.5 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
     |████████████████████████████████| 94 kB 2.9 MB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4
     |████████████████████████████████| 112 kB 8.9 MB/s 
  Attempting uninstall: gunicorn
    Found existing installation: gunicorn 20.0.4
    Uninstalling gunicorn-20.0.4:
      Successfully uninstalled gunicorn-20.0.4
/bin/bash: =4.2b1: No such file or directory
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=54cc4e3d355441625a6782e80d4003a99bfe6361aeca49d1e8374629a1d4ad59
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ngrok authtoken "28TEUMZpSvnTSuNwlXOGhuMqvRq_MaoTNqjiLvN1RvSUeVd9"

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [20]:
!python /content/drive/MyDrive/Colab_Notebooks/2nd_project/web_demo/run.py

2022-05-02 01:19:43.924932: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2022-05-02 01:19:43.925094: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1bef2c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-05-02 01:19:43.925125: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-05-02 01:19:43.926802: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-05-02 01:19:44.162922: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 01:19:44.163836: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1befd40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-05-02 01:19:44

In [ ]:
# -*- coding: utf-8 -*-
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import os
import re
import sys
import pickle
import tensorflow as tf

In [ ]:
# enable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1,
    allow_soft_placement=True,
    device_count={"GPU": 0},
)

In [ ]:
sess = tf.compat.v1.Session(config=config)
graph = tf.compat.v1.get_default_graph()

############################### TODO : 경로 수정하기 ##########################################
sys.path.append(os.path.join(os.getcwd(), "/content/drive/MyDrive/Colab_Notebooks/2nd_project/bert_slot_kor"))
from to_array.bert_to_array import BERTToArray
from models.bert_slot_model import BertSlotModel
from to_array.tokenizationK import FullTokenizer
# pretrained model path
# bert_model_hub_path = os.path.join(os.getcwd(), "bert-module")
# colaboratory에서 실행 시
bert_model_hub_path = '/content/drive/MyDrive/Colab_Notebooks/2nd_project/dataset/model'

# fine-tuned model path
# load_folder_path = os.path.join(os.getcwd(), "model")
# colaboratory에서 실행 시
load_folder_path = "/content/drive/MyDrive/Colab_Notebooks/2nd_project/dataset/saved_model"

# tokenizer vocab file path
vocab_file = os.path.join(bert_model_hub_path, "assets/vocab.korean.rawtext.list")
bert_to_array = BERTToArray(vocab_file)

###########################################################################################

In [ ]:
# 슬롯태깅 모델과 토크나이저 불러오기
tags_to_array_path = os.path.join(load_folder_path, "tags_to_array.pkl")
with open(tags_to_array_path, "rb") as handle:
    tags_to_array = pickle.load(handle)
    slots_num = len(tags_to_array.label_encoder.classes_)

model = BertSlotModel.load(load_folder_path, sess)

tokenizer = FullTokenizer(vocab_file=vocab_file)

bert inputs : [<tf.Tensor 'input_ids:0' shape=(?, ?) dtype=int32>, <tf.Tensor 'input_mask:0' shape=(?, ?) dtype=int32>, <tf.Tensor 'segment_ids:0' shape=(?, ?) dtype=int32>]
init ok


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


slots output : (?, ?, 7)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, None)]       0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, None)]       0                                            
__________________________________________________________________________________________________
KorBertLayer (KorBertLayer)     (None, None, 768)    109693440   input_ids[0][0]                  
                                                                 inpu

In [ ]:
################# TODO : 슬롯 및 선택지 채워넣기 #################
sweetness = ['안 달', '달지 않은', '달지 않고', '드라이', '안 단','안달고','달지않고','달지않은','달달한','달달하고', '달달하지만','달짝지근','단','달콤한', '스위트','많이 단']
body = ['가벼운', '라이트', '가볍', '상쾌한','청량한', '가볍지만','가볍지 않은', '미디엄','진한', '진하고',  '무거운','무겁고', '헤비', '풀', '풀바디', '끈적한', '무겁지만']
sourness = ['안 신', '안 시고',  '시지 않은', '시지 않고','새콤한','상큼한', '시지만', '조금 시큼한','시고','신', '시큼한','시큼하고']
wine_type = ['레드', '화이트', '스파클링', '샴페인', '로제']
price = ['1만정도', '1만원정도', '1만원쯤', '1만원?', '1만?', '1만 이하', '1만원 이하', '1만원이하', '1만이하', '2만정도', '2만원정도', '2만원쯤', '2만원?', '2만?', '2만 이하', '2만원 이하',
 '2만원이하', '2만이하', '3만정도', '3만원정도', '3만원쯤', '3만원?', '3만?', '3만 이하', '3만원 이하', '3만원이하', '3만이하', '4만정도', '4만원정도', '4만원쯤', '4만원?', '4만?', '4만 이하',
 '4만원 이하', '4만원이하', '4만이하', '5만정도', '5만원정도', '5만원쯤', '5만원?', '5만?', '5만 이하', '5만원 이하', '5만원이하', '5만이하', '6만정도', '6만원정도', '6만원쯤', '6만원?', '6만?',
 '6만 이하', '6만원 이하', '6만원이하', '6만이하', '7만정도', '7만원정도', '7만원쯤', '7만원?', '7만?', '7만 이하', '7만원 이하', '7만원이하', '7만이하', '8만정도', '8만원정도', '8만원쯤',
 '8만원?', '8만?', '8만 이하', '8만원 이하', '8만원이하', '8만이하', '9만정도', '9만원정도', '9만원쯤', '9만원?', '9만?', '9만 이하', '9만원 이하', '9만원이하', '9만이하', '10만정도', '10만원정도',
 '10만원쯤', '10만원?', '10만?', '10만 이하', '10만원 이하', '10만원이하', '10만이하', '20만정도', '20만원정도', '20만원쯤', '20만원?', '20만?', '20만 이하', '20만원 이하', '20만원이하', '20만이하',
 '30만정도', '30만원정도', '30만원쯤', '30만원?', '30만?', '30만 이하', '30만원 이하', '30만원이하', '30만이하', '40만정도', '40만원정도', '40만원쯤', '40만원?', '40만?', '40만 이하', '40만원 이하',
 '40만원이하', '40만이하', '50만정도', '50만원정도', '50만원쯤', '50만원?', '50만?', '50만 이하', '50만원 이하', '50만원이하', '50만이하', '60만정도', '60만원정도', '60만원쯤', '60만원?', '60만?',
 '60만 이하', '60만원 이하', '60만원이하', '60만이하', '70만정도', '70만원정도', '70만원쯤', '70만원?', '70만?', '70만 이하', '70만원 이하', '70만원이하', '70만이하', '80만정도', '80만원정도',
 '80만원쯤', '80만원?', '80만?', '80만 이하', '80만원 이하', '80만원이하', '80만이하', '90만정도', '90만원정도', '90만원쯤', '90만원?', '90만?', '90만 이하', '90만원 이하', '90만원이하', '90만이하',
 '100만정도', '100만원정도', '100만원쯤', '100만원?', '100만?', '100만 이하', '100만원 이하', '100만원이하', '100만이하']
 ##############################################################

In [ ]:
# 슬롯 사전
dic = {
    "당도" : sweetness,
    "바디감" : body,
    "산미": sourness,
    "종류" : wine_type,
    "가격" : price
}

# 슬롯 변수명 - 슬롯 이름 pairs
slots = {
    "당도": "당도",
    "바디감": "바디감",
    "산미": "산미",
    "종류": "종류",
    "가격": "가격"
}

# 명령어 설정 ( 챗봇 사용자가 문장 앞에 !를 붙이면 명령어로 인식 )
cmds = {
    "명령어" : ["명령어", "당도", "바디감", "산미", "종류", "가격"],
    "당도" : sweetness,
    "바디감" : body,
    "산미" : sourness,
    "종류" : wine_type,
    "가격" : price
}

In [ ]:
# 슬롯이라고 인식한 토큰을 slot_text에 저장하기
def catch_slot(i, inferred_tags, text_arr, slot_text):
    if not inferred_tags[0][i] == "0":
        word_piece = re.sub("_", " ", text_arr[i])
        slot_text[inferred_tags[0][i]] += word_piece


# 슬롯이 다 채워지면 챗봇 유저에게 확인 메세지 보내기
def check_order_msg(app, slots):
    order = []
    for slot, option in app.slot_dict.items():
        order.append(f"{slots[slot]}: {option}")
    # br 태그는 html에서 줄바꿈을 의미함
    order = "<br />\n".join(set(order))

    message = f"""
        {order} <br />
        위의 와인으로 추천해드릴까요? (예 or 아니오)
        """
    return message

In [ ]:
# 슬롯 초기화 함수
def init_app(app):
    app.slot_dict = {
        "당도": "",
        "바디감": "",
        "산미": "",
        "종류": "",
        "가격": ""
    }
    app.confirm = False

In [ ]:
app = Flask(__name__)
# colaboratory에서 실행 시
run_with_ngrok(app)

In [ ]:
app.static_folder = 'static'

@app.route("/")
def home():
# 사용자가 입력한 슬롯을 저장할 슬롯 사전
    app.slot_dict = {
        "당도": "",
        "바디감": "",
        "산미": "",
        "종류": "",
        "가격": ""
    }
############ TODO : 슬롯이라고 인식할 점수 및 대화에 필요한 변수 설정하기 ############
    # 슬롯으로 인식할 점수 설정하기
    app.score_limit = 0.99
    # 대화에 필요한 변수 설정
    app.confirm = False
    # (필요시) 대화에 필요한 변수 추가하기
##########################################################################
    return render_template("index.html")
 

# 챗봇 사용자가 특정 메세지를 입력했을 때 실행   
@app.route("/get")
def get_bot_response():
    # 사용자가 입력한 문장
    userText = request.args.get('msg').strip() 
    # 명령어 인식 - 사용자가 입력한 문장이 느낌표(!)로 시작할 때
    if userText[0] == "!":
        try:
            li = cmds[userText[1:]]
            message = "<br />\n".join(li)
        except:
            message = "입력한 명령어가 존재하지 않습니다."
        return message


    # 사용자가 입력한 문장을 토큰화
    text_arr = tokenizer.tokenize(userText)
    input_ids, input_mask, segment_ids = bert_to_array.transform([" ".join(text_arr)])

    # 훈련한 슬롯태깅 모델을 사용하여 슬롯 예측
    with graph.as_default():
        with sess.as_default():
            inferred_tags, slots_score = model.predict_slots(
                [input_ids, input_mask, segment_ids], tags_to_array
            )

    # 결과 체크
    print("text_arr:", text_arr)
    print("inferred_tags:", inferred_tags[0])
    print("slots_score:", slots_score[0])

    # inference 결과로 나온 토큰을 저장하는 사전
    slot_text = {k: "" for k in app.slot_dict}

    # 슬롯태깅 실시
    for i in range(0, len(inferred_tags[0])):
        if slots_score[0][i] >= app.score_limit: # 설정한 점수보다 슬롯 점수가 높을 시
            catch_slot(i, inferred_tags, text_arr, slot_text) # 슬롯을 저장하는 함수 실행
        else:
            print("슬롯 인식 중 에러가 발생했습니다.")

############ TODO : 토큰이 알맞은 슬롯으로 인식되게끔 코드 수정 ############
    # 슬롯 사전에 있는 단어와 일치하는지 검증 후 슬롯 사전에 최종으로 저장
    for slot in slot_text:
        if slot_text[slot] in dic[slot]:
            app.slot_dict[slot] = slot_text[slot]
############################################################################
    print(app.slot_dict)

    # 채워지지 않은 슬롯들을 한국어 슬롯 이름으로 변환
    empty_slot = [slots[slot] for slot in app.slot_dict if app.slot_dict[slot] == "" ]


##### TODO : 추출된 슬롯 정보를 가지고 추천 와인까지 출력하는 대화 완성하기 (recommend 함수 적용) #####
    if empty_slot:
        message = ", ".join(empty_slot) + "가 아직 선택되지 않았습니다."
    elif app.confirm == False:
        message = check_order_msg(app, slots)
        app.confirm == True
    else:
        if userText.strip().startswith("예"):
            message = "와인 추천을 진행하겠습니다."
        elif userText.strip().startswith("아니오"):
            message = "다시 주문해주세요."
            init_app(app)
    return message
########################################################################################

In [ ]:
import pandas as pd
import re

data_path = '/content/drive/MyDrive/Colab_Notebooks/2nd_project/dataset/wine_df_all_save.csv' ##### 경로 수정 #####
df = pd.read_csv(data_path) 

In [ ]:
def recommend(slot_dict):
    # 예시) "달지 않은 와인 추천해줘"
    # 1. 맵핑이 필요한 변수 사전 만들기 
    mapping_dict = {"당도": {'안 달' : 0, '달지않은': 0, '달지않고': 0, '드라이': 0, '안 단': 0,'안달고': 0,'달지않고': 0,'달지않은': 0,
                              '달달한': 1,'달달하고': 1, '달달하지만': 1,'달짝지근': 1,
                              '단': 2,'달콤한': 2, '스위트': 2,'많이 단': 2},
                    "바디감": {'가벼운': 0, '라이트': 0, '가볍': 0, '상쾌한': 0,'청량한': 0, '가볍지만': 0,
                              '가볍지 않은': 1, '미디엄': 1,
                              '진한': 2, '진하고': 2,  '무거운': 2,'무겁고': 2, '헤비': 2, '풀': 2, '풀바디': 2, '끈적한': 2, '무겁지만': 2},
                    "산미": {'안 신': 0, '안 시고': 0,  '시지 않은': 0, '시지 않고': 0,
                            '새콤한': 1,'상큼한': 1, '시지만': 1, '조금 시큼한': 1,
                            '시고': 2,'신': 2, '시큼한': 2,'시큼하고': 2}}
    
    # 2. slot_dict의 변수 불러오기 
    tmp_df = df 
    filter_order = ['종류', '금액', '당도', '바디감', '산미'] ##### 필터링 조건 우선순위 수정 #####
    for k, v in sorted(slot_dict.items(), key = lambda i: filter_order.index(i[0])):
        # 예시) k : sweetness, v: 달지 않은

        # 필터링할 와인이 5개 이하면 필터링 중단
        if len(tmp_df) <= 5: 
            break 

        if v:
            # 당도/바디감/산미와 같은 맵핑이 필요한 변수의 경우
            if k in mapping_dict:
                if v in mapping_dict[k]:
                    v = mapping_dict[k][v]
                else: # 맵핑 사전에 없는 슬롯이 출력된 경우, 디폴트값을 2로 지정 ex) 달달구리
                    v = 1


            # 3. df에서 조건 걸어서 찾기 - 필터링 후 tmp_df 변수에 저장
            ##### 필터링 방식 수정 : 여러가지 경우로 생각해보세요 #####
            if k == '종류':
                tmp_df = tmp_df[tmp_df[k] == v]
            elif k == '금액':
                price = re.search('\d+', v).group()
                if '이하' in v:
                    tmp_df = tmp_df[tmp_df[k] <= int(price)*10000]
                ###### elif 정도 in v ######
                else:
                  tmp_df=tmp_df[(tmp_df[k] <= int(price + 1)*10000)&(tmp_df[k] >= int(price - 1)*10000)]
            elif k == '당도':
                if v == 0:
                  tmp_df = tmp_df[tmp_df[k] ==0] # 당도 3으로 exact search를 할 수도 있지만, 3 이상으로 조건을 줄 수도 있습니다
                elif v == 1:
                  tmp_df = tmp_df[(tmp_df[k] >= 1)&(tmp_df[k] <= 2)]
                elif v == 2:
                  tmp_df = tmp_df[tmp_df[k] >=3]
            elif k == '바디감':
                if v == 0:
                  tmp_df = tmp_df[tmp_df[k] <= 2]
                elif v == 1:
                  tmp_df = tmp_df[tmp_df[k] >= 3]
                elif v == 2:
                  tmp_df = tmp_df[tmp_df[k] >= 3]
            elif k == '산미':
                if v == 0:
                  tmp_df = tmp_df[tmp_df[k] <= 2]
                elif v == 1:
                  tmp_df = tmp_df[tmp_df[k] == 3]
                elif v == 2:
                  tmp_df = tmp_df[tmp_df[k] >= 4]
        
    # 4. 걸러진 와인 리스트 추천순위 기준 정렬
    wine_list = tmp_df.sort_values('평점지수')

    ##### 현재 코드는 필터링된 와인 중 추천 순위 1위 와인 출력, 필요한 요소들을 리턴하도록 수정 #####
    return wine_list.iloc[0]['이름']

In [ ]:
mapping_dict = {"당도": {'안 달' : 0, '달지않은': 0, '달지않고': 0, '드라이': 0, '안 단': 0,'안달고': 0,'달지않고': 0,'달지않은': 0,
                              '달달한': 1,'달달하고': 1, '달달하지만': 1,'달짝지근': 1,
                              '단': 2,'달콤한': 2, '스위트': 2,'많이 단': 2},
                    "바디감": {'가벼운': 0, '라이트': 0, '가볍': 0, '상쾌한': 0,'청량한': 0, '가볍지만': 0,
                              '가볍지 않은': 1, '미디엄': 1,
                              '진한': 2, '진하고': 2,  '무거운': 2,'무겁고': 2, '헤비': 2, '풀': 2, '풀바디': 2, '끈적한': 2, '무겁지만': 2},
                    "산미": {'안 신': 0, '안 시고': 0,  '시지 않은': 0, '시지 않고': 0,
                            '새콤한': 1,'상큼한': 1, '시지만': 1, '조금 시큼한': 1,
                            '시고': 2,'신': 2, '시큼한': 2,'시큼하고': 2}}

In [ ]:
tmp_df = df 
filter_order = ['종류', '금액', '당도', '바디감', '산미'] ##### 필터링 조건 우선순위 수정 #####

In [ ]:
for k, v in sorted(example.items(), key = lambda i: filter_order.index(i[0])):
        # 예시) k : sweetness, v: 달지 않은

        # 필터링할 와인이 5개 이하면 필터링 중단
        if len(tmp_df) <= 5: 
            break 

        if v:
            # 당도/바디감/산미와 같은 맵핑이 필요한 변수의 경우
            if k in mapping_dict:
                if v in mapping_dict[k]:
                    v = mapping_dict[k][v]
                else: # 맵핑 사전에 없는 슬롯이 출력된 경우, 디폴트값을 2로 지정 ex) 달달구리
                    v = 1


            # 3. df에서 조건 걸어서 찾기 - 필터링 후 tmp_df 변수에 저장
            ##### 필터링 방식 수정 : 여러가지 경우로 생각해보세요 #####
            if k == '종류':
                tmp_df = tmp_df[tmp_df[k] == v]
            elif k == '금액':
                price = re.search('\d+', v).group()
                if '이하' in v:
                    tmp_df = tmp_df[tmp_df[k] <= int(price)*10000]
                ###### elif 정도 in v ######
                else:
                  tmp_df=tmp_df[(tmp_df[k] <= int(price + 1)*10000)&(tmp_df[k] >= int(price - 1)*10000)]
            elif k == '당도':
                if v == 0:
                  tmp_df = tmp_df[tmp_df[k] ==0] # 당도 3으로 exact search를 할 수도 있지만, 3 이상으로 조건을 줄 수도 있습니다
                elif v == 1:
                  tmp_df = tmp_df[(tmp_df[k] >= 1)&(tmp_df[k] <= 2)]
                elif v == 2:
                  tmp_df = tmp_df[tmp_df[k] >=3]
            elif k == '바디감':
                if v == 0:
                  tmp_df = tmp_df[tmp_df[k] <= 2]
                elif v == 1:
                  tmp_df = tmp_df[tmp_df[k] >= 3]
                elif v == 2:
                  tmp_df = tmp_df[tmp_df[k] >= 3]
            elif k == '산미':
                if v == 0:
                  tmp_df = tmp_df[tmp_df[k] <= 2]
                elif v == 1:
                  tmp_df = tmp_df[tmp_df[k] == 3]
                elif v == 2:
                  tmp_df = tmp_df[tmp_df[k] >= 4]

In [ ]:
tmp_df

,Unnamed: 0,이름,영문이름,종류,나라,생산지,제조사,포도품종,알코올도수,금액,...,산미,당도,바디감,탄닌,인기순위,추천순위,구매가능여부,주소,이미지주소,평점지수
2,2,빌라 안티노리 키안티 클라시코 리제르바,VILLA ANTINORI CHIANTI CLASSICO RISERVA,레드,이탈리아,"토스카나, 키안티 클라시코",안티노리,"산지오베제 90%, 카베르네 소비뇽 10%",14.0,39000,...,3,0,3,4,7,117,구매 불가,https://www.winenara.com/shop/product/product_...,https://www.winenara.com/uploads/product/550/1...,4.867148
15,15,캔달잭슨 빈트너스 리저브 피노누아,KENDALL JACKSON VINTNER`S RESERVE PINOT NOIR,레드,미국,"미국, 캘리포니아",잭슨 패밀리 와인,피노누아(PINOT NOIR) 100%,14.5,45000,...,3,0,4,3,26,659,구매 예약,https://www.winenara.com/shop/product/product_...,https://www.winenara.com/uploads/product/550/2...,4.531818
34,34,베네가쭈 델라 카사,VENEGAZZU DELLA CASA,레드,이탈리아,베네토,베네가쭈,"카베르네 소비뇽(CABERNET SAUVIGNON) 60%, 메를로(MERLOT) ...",13.0,35000,...,3,0,3,3,49,420,구매 예약,https://www.winenara.com/shop/product/product_...,https://www.winenara.com/uploads/product/550/2...,4.989049
36,36,닥터 린드만 빈45 카베르네 소비뇽,LINDEMAN'S BIN 45 CABERNET SAUVIGNON,레드,호주,뉴 사우스 웨일즈 & 빅토리아,린드만 (Lindeman’s),까베르네 소비뇽 (Cabernet Sauvignon),13.5,20000,...,3,0,3,3,51,604,구매 예약,https://www.winenara.com/shop/product/product_...,https://www.winenara.com/uploads/product/550/1...,4.185330
38,38,본테라 에퀴녹스 레드,BONTERRA EQUINOX RED,레드,미국,"캘리포니아, 홉랜드, 멘도치노 카운티",본테라 / 콘차 이 토로,"쁘띠 쉬라 33%, 메를로 29%, 쉬라 16%, 진판델 14%, 카베르네 소비뇽 ...",14.5,50000,...,3,0,4,4,54,515,구매 예약,https://www.winenara.com/shop/product/product_...,https://www.winenara.com/uploads/product/550/1...,4.479751
40,40,베이컨,BACON,레드,미국,"미국, 캘리포니아",Guarachi Wine Partners,"시라(SYRAH) 35% , 그르나슈(GRENACHE) 25% , 무르베드르(MOU...",15.0,35000,...,3,0,5,3,57,9,구매 예약,https://www.winenara.com/shop/product/product_...,https://www.winenara.com/uploads/product/550/2...,4.574173
42,42,바독 피노누아,BAR DOG PINOT NOIR,레드,미국,캘리포니아,바 독,피노 누아,13.5,32000,...,3,0,3,0,59,210,구매 예약,https://www.winenara.com/shop/product/product_...,https://www.winenara.com/uploads/product/550/1...,4.370454
45,45,핀카 비야크레세스 프루노,FINCA VILLACRECES PRUNO,레드,스페인,NaN,핀카 비야크레세스 (Finca Villacreces),NaN,14.0,45000,...,3,0,4,3,63,451,구매 불가,https://www.winenara.com/shop/product/product_...,https://www.winenara.com/uploads/product/550/1...,4.763016
46,46,로스 인토카블레스 말벡,LOS INTOCABLES MALBEC,레드,아르헨티나,아르헨티나,핀카 라스 모라스 Finca Las Moras (Penaflor),말벡(MALBEC) 100%,14.0,21900,...,3,0,4,3,64,329,구매 예약,https://www.winenara.com/shop/product/product_...,https://www.winenara.com/uploads/product/550/2...,4.761156
48,48,유 원 프리미티보 디 만두리아,U-ONE PRIMITIVO DI MANDURIA,레드,이탈리아,"풀리아, 만두리아",쉥크,프리미티보 100%,14.5,29000,...,3,0,4,3,66,599,구매 불가,https://www.winenara.com/shop/product/product_...,https://www.winenara.com/uploads/product/550/1...,4.957437


In [ ]:
example = {'당도': '달지않고', '바디감': '진한', '산미': '새콤한', '종류': '레드', '금액': '5만원이하'}
recommend(example)

IndexError: ignored

In [ ]:
if __name__ == "__main__":
    example = {
            "sweetness": "달지 않은",
            "body": "",
            "sourness": "",
            "wine_type": "",
            "price": ""
        }
    print(recommend(example))